In [ ]:
import os
import numpy as np
import pandas as pd

range_resolution = 0.0214
n_angle_bins = 64
POWER_THRESH = 1e2

RUNS = [
    #sit
    (r"C:\Users\USER\Desktop\DataCollection\Sitting\Run1-SmallRoomAverageNoise\OriginalRunIQ.csv", "sit",100,66),
    (r"C:\Users\USER\Desktop\DataCollection\Sitting\Run2-Lab\OriginalRunIQ.csv",  "sit",100,166),
    (r"C:\Users\USER\Desktop\DataCollection\Sitting\run3bed\run3sit.csv", "sit",0,66),
    #stand
    (r"C:\Users\USER\Desktop\DataCollection\Standing\StandingRun1SmallRoom\Original.csv",  "stand",260,326),
    (r"C:\Users\USER\Desktop\DataCollection\Standing\StandingRun2CompLab\stand2run.csv", "stand",100,166),
    (r"C:\Users\USER\Desktop\DataCollection\Standing\standrun3bedroom\run3bed.csv",  "stand",0,66),
    #lying
    (r"C:\Users\USER\Desktop\DataCollection\Lying\Run1-SmallRoomCouch\Original.csv",  "lying",420,486),
    (r"C:\Users\USER\Desktop\DataCollection\Lying\run2chairLab\run2lab.csv", "lying",60,126),
    (r"C:\Users\USER\Desktop\DataCollection\Lying\run3bed\run3lying.csv",  "lying",0,66),
    #walk
    (r"C:\Users\USER\Desktop\DataCollection\Walking\WalkingRun1SmallRoom\Original.csv",  "walk",100,166),
    (r"C:\Users\USER\Desktop\DataCollection\Walking\walkingrun2-lab\walk2.csv", "walk",10,76),
    (r"C:\Users\USER\Desktop\DataCollection\Walking\walk3run\walk3bed.csv",  "walk",0,66),
    #fall
    (r"C:\Users\USER\Desktop\DataCollection\Falling\FallingRun1SmallLivingRoomToCouch\Original.csv",  "fall",100,166),
    (r"C:\Users\USER\Desktop\DataCollection\Falling\fallingLab2\fall2lab.csv", "fall",20,86),
    (r"C:\Users\USER\Desktop\DataCollection\Falling\fall3bed\bedfall3.csv",  "fall",0,66)
]

SAVE_DIR_XYZ = r"C:\Users\USER\precomputed_pointclouds\xyz"           
SAVE_DIR_MAG = r"C:\Users\USER\precomputed_pointclouds\magnitude"
os.makedirs(SAVE_DIR_XYZ, exist_ok=True)
os.makedirs(SAVE_DIR_MAG, exist_ok=True)

def _unique_path(base_dir, base_name):
    path = os.path.join(base_dir, base_name)
    counter = 2
    while os.path.exists(path):
        root, ext = os.path.splitext(base_name)
        base_name = f"{root}({counter}){ext}"
        path = os.path.join(base_dir, base_name)
        counter += 1
    return path

def _range_fft_and_points(df, num_frames_to_keep=66, with_intensity=False):
    #frame number handling + taking middle 66 frames
    unique_frames = sorted(df["frame"].unique())
    total_frames = len(unique_frames)

    if total_frames < num_frames_to_keep:
        print(f"[WARN] CSV only has {total_frames} frames (< {num_frames_to_keep}) — keeping all.")
        frames_to_use = unique_frames
    else:
        mid_start = (total_frames - num_frames_to_keep) // 2
        frames_to_use = unique_frames[mid_start: mid_start + num_frames_to_keep]

    df = df[df["frame"].isin(frames_to_use)]

    #reindex frames
    frame_map = {old: new for new, old in enumerate(sorted(df["frame"].unique()))}
    df["frame"] = df["frame"].map(frame_map)

    n_frames  = int(df["frame"].max()) + 1
    n_rx      = int(df["rx"].max()) + 1
    n_chirps  = int(df["chirp"].max()) + 1
    n_samples = int(df["sample"].max()) + 1

    pointclouds = []
    sin_axis = np.linspace(-1, 1, n_angle_bins, dtype=np.float32)#compute sin axis for Angle of Arrival

    #process each frame
    for f in range(n_frames):
        fdf = df[df["frame"] == f]
        frame_data = np.zeros((n_rx, n_samples, n_chirps), dtype=np.complex64)

        # fast vectorized fill
        rr  = fdf["rx"].to_numpy(np.int64)
        ss  = fdf["sample"].to_numpy(np.int64)
        cc  = fdf["chirp"].to_numpy(np.int64)
        vals = fdf["I"].to_numpy(np.float32) + 1j * fdf["Q"].to_numpy(np.float32) if "I" in fdf.columns else fdf["ival"].to_numpy(np.float32) + 1j * fdf["qval"].to_numpy(np.float32)
        frame_data[rr, ss, cc] = vals

        #apply hanning window and range fft
        window = np.hanning(n_samples).astype(np.float32)
        windowed = frame_data * window[:, None]
        range_fft = np.fft.fft(windowed, axis=1)[:, : n_samples // 2, :]

        #building points
        pts = []
        for r in range(range_fft.shape[1]):
            rng = r * range_resolution
            for d in range(range_fft.shape[2]):
                signal = range_fft[:, r, d]
                if np.abs(signal).max() < POWER_THRESH or n_rx < 3: #thresholding
                    continue

                rx1, rx2, rx3 = signal[0], signal[1], signal[2]

                #applying azmuith and elevation transformations using 3 recieving antennas
                az_signal = np.array([rx2, rx1], dtype=np.complex64)
                az_fft = np.fft.fftshift(np.fft.fft(az_signal, n=n_angle_bins))
                az_idx = int(np.argmax(np.abs(az_fft)))
                sin_theta = float(np.clip(sin_axis[az_idx], -1.0, 1.0))
                theta = np.arcsin(sin_theta)

                el_signal = np.array([rx2, rx3], dtype=np.complex64)
                el_fft = np.fft.fftshift(np.fft.fft(el_signal, n=n_angle_bins))
                el_idx = int(np.argmax(np.abs(el_fft)))
                sin_phi = float(np.clip(sin_axis[el_idx], -1.0, 1.0))
                phi = np.arcsin(sin_phi)

                #converting polar coords to cartesian x,y,z
                x = rng * np.cos(phi) * np.sin(theta)
                y = rng * np.cos(phi) * np.cos(theta)
                z = rng * np.sin(phi)

                if with_intensity:
                    mag = float(np.abs(signal).max())
                    pts.append((x, y, z, mag))
                else:
                    pts.append((x, y, z))

        if with_intensity:
            pointclouds.append(np.array(pts, dtype=np.float32) if pts else np.zeros((0, 4), dtype=np.float32))
        else:
            pointclouds.append(np.array(pts, dtype=np.float32) if pts else np.zeros((0, 3), dtype=np.float32))

    return pointclouds, total_frames

def read_and_filter_pointclouds_xyz(path, num_frames_to_keep=66):
    if not os.path.isfile(path):
        raise FileNotFoundError(f"CSV not found: {path}")

    df = pd.read_csv(path)

    #rename cols
    df = df.rename(columns={
        "Frame Index": "frame",
        "Chirp Index": "chirp",
        "Sample Index": "sample",
        "Antenna Index": "rx",
        "I": "ival",
        "Q": "qval"
    })

    #ensure all cols are numeric
    for col in ["frame", "chirp", "sample", "rx"]:
        df[col] = pd.to_numeric(df[col], errors="raise").astype(int)
    for col in ["ival", "qval"]:
        df[col] = pd.to_numeric(df[col], errors="raise").astype(np.float32)

    pcs, total_frames = _range_fft_and_points(df, num_frames_to_keep=num_frames_to_keep, with_intensity=False)
    print(f"[INFO] {path}: kept {len(pcs)} frames out of {total_frames} total.")
    return pcs

def read_and_filter_pointclouds_mag(path, num_frames_to_keep=66):
    if not os.path.isfile(path):
        raise FileNotFoundError(f"CSV not found: {path}")

    df = pd.read_csv(path).rename(columns={
        "Frame Index": "frame",
        "Chirp Index": "chirp",
        "Sample Index": "sample",
        "Antenna Index": "rx",
        "I": "ival",
        "Q": "qval"
    })

    # Types
    for col in ["frame", "chirp", "sample", "rx"]:
        df[col] = pd.to_numeric(df[col], errors="raise").astype(int)
    for col in ["ival", "qval"]:
        df[col] = pd.to_numeric(df[col], errors="raise").astype(np.float32)

    pcs, total_frames = _range_fft_and_points(df, num_frames_to_keep=num_frames_to_keep, with_intensity=True)
    print(f"[INFO] {os.path.basename(path)}: kept {len(pcs)} frames of {total_frames}")
    return pcs  #list of (Ni, 4): [x,y,z,intensity]

#Processing and saving all runs (XYZ-only)
for path, label, _, _ in RUNS:  
    stem = os.path.splitext(os.path.basename(path))[0]
    parent = os.path.basename(os.path.dirname(path))
    pcs = read_and_filter_pointclouds_xyz(path, num_frames_to_keep=66)

    save_name = f"{parent}_{stem}_{label}_{len(pcs)}frames.npz"
    save_path = _unique_path(SAVE_DIR_XYZ, save_name)

    pcs_obj = np.array(pcs, dtype=object)  # ragged list → object array
    np.savez_compressed(save_path, pcs=pcs_obj, label=label, n_frames=len(pcs))
    print(f"[SAVED xyz] {label}: {len(pcs)} frames → {save_path}")

####repeat process including magnitude alongside each x,y,z pair

#loop to stop as n=4, with magnitude as feature
for path, label, _, _ in RUNS:
    parent = os.path.basename(os.path.dirname(path))
    stem = os.path.splitext(os.path.basename(path))[0]

    pcs = read_and_filter_pointclouds_mag(path, num_frames_to_keep=66)  # list of (Ni,4)

    save_name = f"{parent}_{stem}_{label}_{len(pcs)}frames_intensity.npz"
    save_path = _unique_path(SAVE_DIR_MAG, save_name)

    pcs_obj = np.array(pcs, dtype=object)
    np.savez_compressed(
        save_path,
        pcs=pcs_obj,         
        label=label,
        n_frames=len(pcs),
        has_intensity=True,   #magnitude flag
        cols=np.array(["x","y","z","intensity"])
    )
    print(f"[SAVED mag] {label}: {len(pcs)} frames → {save_path}")


    
import random
import matplotlib.pyplot as plt

def _clean_mag_pc(pc):
  
    if pc is None:
        return np.zeros((0,4), dtype=np.float32)
    arr = np.asarray(pc)
    #handling n=3, padding if nomag present
    if arr.ndim == 1:
        arr = np.atleast_2d(arr)
    if arr.shape[1] == 3:
        arr = np.column_stack([arr, np.zeros((arr.shape[0],), dtype=arr.dtype)])
    if arr.ndim != 2 or arr.shape[1] != 4:
        return np.zeros((0,4), dtype=np.float32)
    arr = arr.astype(np.float32, copy=False)
    mask = np.isfinite(arr).all(axis=1)
    arr = arr[mask]
    return arr

#handling of extreme outliers distorting colour scale
def _percentile_clip(values, low=1, high=99):
    if values.size == 0:
        return 0.0, 1.0
    vmin = float(np.percentile(values, low))
    vmax = float(np.percentile(values, high))
    if not np.isfinite(vmin) or not np.isfinite(vmax) or vmin == vmax:
        vmax = vmin + 1.0
    return vmin, vmax


#plotting num_frames from npz file for visualisation
def plot_magnitude_pointclouds(npz_path, num_frames=5, min_points=10, thresh_pkd=None, seed=0, save_png_dir=None):
    d = np.load(npz_path, allow_pickle=True)
    pcs = list(d["pcs"])
    label = str(d["label"]) if "label" in d.files else "unknown"

    cleaned = []
    for i, pc in enumerate(pcs):
        arr = _clean_mag_pc(pc)

        #Apply threshold
        if thresh_pkd is not None:
            arr = arr[arr[:, 3] >= thresh_pkd]

        if arr.shape[0] >= min_points:
            cleaned.append((i, arr))

    if not cleaned:
        print(f"[WARN] No valid frames (≥{min_points} pts) in {os.path.basename(npz_path)}")
        return

    random.seed(seed)
    sample = random.sample(cleaned, k=min(num_frames, len(cleaned)))

    #creating a shared axis cube for more consistent viewing
    all_pts = np.vstack([arr[:, :3] for _, arr in sample])
    mins = all_pts.min(axis=0); maxs = all_pts.max(axis=0)
    center = (mins + maxs) / 2.0
    span = float(np.max(maxs - mins)) or 1.0

    #saving plots
    if save_png_dir:
        os.makedirs(save_png_dir, exist_ok=True)

    for idx, arr in sample:
        x, y, z, mag = arr[:,0], arr[:,1], arr[:,2], arr[:,3]
        vmin, vmax = _percentile_clip(mag, 2, 98)  # robust color limits

        fig = plt.figure(figsize=(6,5))
        ax = fig.add_subplot(111, projection='3d')
        sc = ax.scatter(x, y, z, c=mag, s=4, vmin=vmin, vmax=vmax)
        ax.set_title(f"{os.path.basename(npz_path)} | Frame {idx} | {label}")
        ax.set_xlabel("X (m)"); ax.set_ylabel("Y (m)"); ax.set_zlabel("Z (m)")
        ax.set_xlim(center[0]-span/2, center[0]+span/2)
        ax.set_ylim(center[1]-span/2, center[1]+span/2)
        ax.set_zlim(center[2]-span/2, center[2]+span/2)
        cbar = plt.colorbar(sc, ax=ax)
        cbar.set_label("Magnitude")
        plt.tight_layout()

        if save_png_dir:
            out_name = f"{os.path.splitext(os.path.basename(npz_path))[0]}_frame{idx}.png"
            out_path = os.path.join(save_png_dir, out_name)
            plt.savefig(out_path, dpi=200)
            plt.close(fig)
            print(f"[PLOT SAVED] {out_path}")
        else:
            plt.show()